In [96]:
import pandas as pd
import chardet
import os
import re
from datetime import datetime

pd.set_option("display.max_columns", None)

### Functions

In [97]:
# 📌 Définir les dossiers
DOSSIER_VENDOR = "../source/vendor_central_mois"
DOSSIER_AMVISOR = "../source/amvisor_mois"
DOSSIER_NET_PPM = "../source/vendor_central_netppm"
DOSSIER_DATA = "../data"

# Fonction pour extraire la date (YYYY-MM) d'un fichier
def extraire_mois_annee(nom_fichier):
    # Format AMVisor (YYYY-MM)
    match_amvisor = re.search(r"(\d{4})-(\d{2})", nom_fichier)
    if match_amvisor:
        return match_amvisor.group(0)
    
    # Format Vendor Central (JJ-MM-YYYY_JJ-MM-YYYY)
    match_vendor = re.search(r"(\d{2})-(\d{2})-(\d{4})_\d{2}-\d{2}-\d{4}", nom_fichier)
    if match_vendor:
        return f"{match_vendor.group(3)}-{match_vendor.group(2)}"  

    return None

# Fonction pour récupérer les fichiers d'un mois donné et vérifier leur présence
def get_files(annee_mois):
    fichiers_vendor = os.listdir(DOSSIER_VENDOR)
    fichiers_amvisor = os.listdir(DOSSIER_AMVISOR)
    fichiers_net_ppm = os.listdir(DOSSIER_NET_PPM)

    # Filtrer les fichiers correspondant au mois donné
    fichiers_trouves = [
        os.path.join(DOSSIER_AMVISOR, f) for f in fichiers_amvisor if extraire_mois_annee(f) == annee_mois
    ] + [
        os.path.join(DOSSIER_VENDOR, f) for f in fichiers_vendor if extraire_mois_annee(f) == annee_mois
    ] + [
        os.path.join(DOSSIER_NET_PPM, f) for f in fichiers_net_ppm if extraire_mois_annee(f) == annee_mois
    ]

    # Vérification : on doit avoir exactement 3 fichiers (Export, Fabrication, Approvisionnement)
    if len(fichiers_trouves) < 4:
        print(f"❌ Fichiers manquants pour {annee_mois} : trouvés {len(fichiers_trouves)}/3.") # raise FileNotFoundError
    if len(fichiers_trouves) == 2:
        fichiers_trouves = fichiers_trouves

    return fichiers_trouves
    
# Fonction de processing des fichiers Amvisor
def process_amvisor(path):

    # Get columns
    columns_main = ["ASIN", 'Item no.', "EAN", "Item"]
    columns_overall = ["Extras", "Size", "Description Item", "Cat. 1", "Cat. 2", "Cat. 3", "Cat. 4", "Visible", "Title Content", "Variations", "Brand store URL", "Images", "Videos", "AI summary", "Code", "Catalogue"]
    columns_ads = ["Ads Impressions CM", "Ads Clicks CM", "Ads CTR CM", "Ads Units 14d CM", "Ads Costs CM", "Ads RoAS CM", "Ads CVR CM"]
    columns_sales = ["Sell-out CM", "Sell-out PM", 'Total sell-out CM', 'Total sell-out PM', "Units CM", "Units PM", "SRP", "Margin", "Replacements CM", "Replacements PM"] #, "Revenue CM", "Revenue PM"
    columns_stocks = ['Stock', 'Stock value', 'Total stock', 'Total stock value']
    columns_state = ["Reviews", "Avg. stars", "Stars",'Coverage', 'Rank 1', 'Rank 2', 'Buy Box', 'Buy Box PM', 'Days not Buy Box', "Views CM", "Views PM", "CVR CM", "CVR PM"]
    all_columns = columns_main + columns_overall + columns_ads + columns_sales + columns_stocks + columns_state

    # Return empty DataFrame if no file found
    if not path:
        return pd.DataFrame(columns=all_columns)

    # Load Amvisor
    df = pd.read_csv(path, sep=None, engine='python', encoding="ISO-8859-1", dtype={"EAN": str})
    df = df.dropna(subset=["ASIN"])
    
    # Transform columns
    colonnes_a_convertir = columns_ads + columns_sales + columns_stocks + columns_state
    colonnes_a_convertir = [col for col in colonnes_a_convertir if col != "Buy Box"]
    df[colonnes_a_convertir] = df[colonnes_a_convertir].replace({"€": "", "%": "", "\u202f": "", ",": "."}, regex=True).astype(float)

    # Select data
    all_columns = [col for col in all_columns if col in df.columns]
    df = df[all_columns]

    return df

# Fonction de processing des fichiers Vendor Central
def process_vendor_central(path1, path2):

    # Load data
    df_fab = pd.read_csv(path1, skiprows=1)
    df_app = pd.read_csv(path2, skiprows=1)

    # Select columns
    columns = ["ASIN", "Nom du produit", "Marque", "COGS expédié", "COGS expédié – Période antérieure (%)", "COGS expédié – Même période l'année dernière (%)", "Unités expédiées", "Unités expédiées – Période antérieure (%)", "Unités expédiées – Même période l'année dernière (%)", "Retours client", "Retours du client – Période antérieure (%)", "Retours du client – Même période l'année dernière (%)"]
    df_fab = df_fab[columns]
    df_app = df_app[columns]
    df_fab['fab_or_app'] = "1_fab"
    df_app['fab_or_app'] = "2_app"

    # Merge data
    df = pd.concat([df_fab, df_app])

    # Convert data
    colonnes_a_convertir = [
        "COGS expédié", "COGS expédié – Période antérieure (%)", "COGS expédié – Même période l'année dernière (%)",
        "Unités expédiées", "Unités expédiées – Période antérieure (%)", "Unités expédiées – Même période l'année dernière (%)",
        "Retours client", "Retours du client – Période antérieure (%)", "Retours du client – Même période l'année dernière (%)"
    ]
    df[colonnes_a_convertir] = df[colonnes_a_convertir].replace({"€": "", "%": "", "\u202f": "", ",": "."}, regex=True).astype(float)

    # Remove duplicates based on COGS
    df = df.sort_values(by=["ASIN", "COGS expédié", "fab_or_app"], ascending=[True, False, True])
    df = df.drop_duplicates(subset="ASIN", keep="first")

    # Rename columns
    columns = ["ASIN", "Nom du produit", "Marque", "COGS", "COGS LM (%)", "COGS SPLY (%)", "Unités", "Unités LM (%)", "Unités SPLY (%)", "Retours", "Retours LM (%)", "Retours SPLY (%)", "Source (Fabrication ou Approvisionnement)"]
    df.columns = columns

    # Compute before evols
    # df["COGS LM"] = (df["COGS"] / (1 + df["COGS LM (%)"] / 100)).round(2)
    # df["COGS SPLY"] = (df["COGS"] / (1 + df["COGS SPLY (%)"] / 100)).round(2)
    # df["Unités LM"] = (df["Unités"] / (1 + df["Unités LM (%)"] / 100)).round().fillna(0).astype(int)
    # df["Retours LM"] = df["Retours"] / (1 + df["Retours LM (%)"] / 100)

    return df

# Fonction de processing des fichiers Net PPM
def process_net_ppm(path):

    # Load data
    df = pd.read_csv(path, skiprows=1)

    # Select columns
    columns = ["ASIN", "PPM nette", "PPM nette – Même période l'année dernière (bps)"]
    df = df[columns]

    # Convert data
    colonnes_a_convertir = ["PPM nette", "PPM nette – Même période l'année dernière (bps)"]
    df[colonnes_a_convertir] = df[colonnes_a_convertir].replace({"€": "", "%": "", "\u202f": "", ",": "."}, regex=True).astype(float)

    # Compute before evols
    df["Net PPM SPLY"] = df["PPM nette"] / (1 + df["PPM nette – Même période l'année dernière (bps)"] / 10000)
    df.drop(columns=["PPM nette – Même période l'année dernière (bps)"], inplace=True)

    # Rename columns
    columns = ["ASIN", "Net PPM", "Net PPM SPLY"]
    df.columns = columns

    return df


# Fonction de merge des fichiers Vendor Central et Amvisor
def process_amazon(df_amvisor, df_vendor_central, df_net_ppm):

    df_amvisor['is_in_amvisor'] = True
    df_vendor_central['is_in_vendor_central'] = True

    # Merge data
    df = pd.merge(df_vendor_central, df_amvisor, how="outer", left_on="ASIN", right_on="ASIN")
    print(f"amvisor : {len(df_amvisor)} | vendor_central : {len(df_vendor_central)} | merged : {len(df)}")

    # Fill missing flags with False
    df['is_in_amvisor'] = df['is_in_amvisor'].fillna(False)
    df['is_in_vendor_central'] = df['is_in_vendor_central'].fillna(False)

    # Merge with net ppm
    df = pd.merge(df, df_net_ppm, how="left", on="ASIN")
    
    return df


# Fonction de processing des fichiers Net PPM
def process_stocks(path="../source/amvisor_stocks/hama_stock_20240101_20250331.csv"):
    print("-- Processing stocks ...")

    # Load data
    df = pd.read_csv(path, skiprows=0, low_memory=False)
    cols_to_keep = ['ASIN'] + [col for col in df.columns if 'stock units' in col.lower()]
    df = df[cols_to_keep]

    # Clean columns: remove \r\n and split into 'date' and 'metric'
    df.columns = df.columns.str.replace(r'\r\n', ' ', regex=True)

    # Melt the DataFrame to long format
    dt = df.melt(id_vars='ASIN', var_name='date_metric', value_name='value')

    # Split date and metric
    dt[['date', 'metric']] = dt['date_metric'].str.extract(r'(\d{2}/\d{2}/\d{4}) (.*)')
    dt['date'] = pd.to_datetime(dt['date'], format='%m/%d/%Y')

    stock_units_df = dt[dt['metric'] == 'Stock units']
    stock_units_df = stock_units_df[stock_units_df['ASIN'].notna()]
    stock_units_df.rename(columns={"date": "extract_date"}, inplace=True)
    stock_units_df = stock_units_df.pivot(index='ASIN', columns='extract_date', values='value')
    stock_units_df = stock_units_df.dropna(subset=stock_units_df.columns.difference(['ASIN']), how='all')

    total_stock_units_df = dt[dt['metric'] == 'Stock units']
    total_stock_units_df = total_stock_units_df[total_stock_units_df['ASIN'].notna()]
    total_stock_units_df.rename(columns={"date": "extract_date"}, inplace=True)
    total_stock_units_df = total_stock_units_df.pivot(index='ASIN', columns='extract_date', values='value')
    total_stock_units_df = total_stock_units_df.dropna(subset=total_stock_units_df.columns.difference(['ASIN']), how='all')

    # Export to CSV
    total_stock_units_df.to_csv(f"{DOSSIER_DATA}/total_stock_units.csv", index=False)
    stock_units_df.to_csv(f"{DOSSIER_DATA}/stock_units.csv", index=False)

    return stock_units_df, total_stock_units_df


# Processing loop
def processing(dates, DOSSIER_DATA):
    all_data = []

    for date in dates:
        print(f"-- Processing {date} ...")

        # Get files and process it
        files = get_files(date)
        fichier_amvisor = next((f for f in files if "Export Hama" in f), None)
        fichier_fabrication = next((f for f in files if "Fabrication" in f), None)
        fichier_approvisionnement = next((f for f in files if "Approvisionnement" in f), None)
        fichier_net_ppm = next((f for f in files if "PPM_nette" in f), None)
        amvisor = process_amvisor(fichier_amvisor)
        vendor_central = process_vendor_central(fichier_fabrication,fichier_approvisionnement)
        net_ppm = process_net_ppm(fichier_net_ppm)
        data = process_amazon(amvisor, vendor_central, net_ppm)

        data["extract_date"] = date
        data["annee"] = data["extract_date"].str[:4].astype(int)
        data["mois"] = data["extract_date"].str[5:7].astype(int)
        data.to_csv(f"{DOSSIER_DATA}/data_{date}.csv")
        all_data.append(data)

    # Concatenate all data
    final_df = pd.concat(all_data, ignore_index=True)

    # Further computations

    ## -- COGS SPLY
    final_df["extract_date_LY"] = pd.to_datetime(final_df["extract_date"], format="%Y-%m").dt.to_period("M") + 12
    dt = final_df[["ASIN", "extract_date_LY", "COGS"]].copy()
    dt["extract_date_LY"] = dt["extract_date_LY"].astype(str)
    dt.rename(columns={"COGS": "COGS SPLY (€)", "extract_date_LY": "extract_date"}, inplace=True)
    final_df = final_df.merge(dt, on=["ASIN", "extract_date"], how="left")
    final_df = final_df.drop(columns=["extract_date_LY"])
    ## -- COGS diff SPLY
    final_df["COGS diff SPLY (€)"] = final_df["COGS"] - final_df["COGS SPLY (€)"]

    ## -- New categories
    new_cat = pd.read_excel("../source/adhoc/Sales part Cross reference IFS   AMAZON vers3 sans les doublons au 18.05.25 avec categories DE .xlsx")
    new_cat = new_cat[['Ref AMAZON', 'Cat DE 1 ', "Cat DE 2"]]
    new_cat.columns = ['ASIN', 'Category 1', "Category 2"]
    final_df = final_df.merge(new_cat, on=["ASIN"], how="left")

    process_stocks()
    
    
    final_df.to_csv(f"{DOSSIER_DATA}/all_data.csv", index=False)
    return final_df

### Processing

In [98]:
dates = ["2025-03", "2025-02", "2025-01", "2024-12","2024-11","2024-10","2024-09","2024-08","2024-07","2024-06","2024-05","2024-04","2024-03","2024-02","2024-01"]
data = processing(dates, DOSSIER_DATA)

-- Processing 2025-03 ...
amvisor : 2715 | vendor_central : 1274 | merged : 2981
-- Processing 2025-02 ...
amvisor : 2583 | vendor_central : 1252 | merged : 2834
-- Processing 2025-01 ...
amvisor : 2225 | vendor_central : 1294 | merged : 2533
-- Processing 2024-12 ...
amvisor : 2234 | vendor_central : 1279 | merged : 2532
-- Processing 2024-11 ...
amvisor : 2251 | vendor_central : 1269 | merged : 2532
-- Processing 2024-10 ...
amvisor : 2205 | vendor_central : 1270 | merged : 2494
-- Processing 2024-09 ...
❌ Fichiers manquants pour 2024-09 : trouvés 3/3.
amvisor : 0 | vendor_central : 1254 | merged : 1254
-- Processing 2024-08 ...
amvisor : 2286 | vendor_central : 1307 | merged : 2562
-- Processing 2024-07 ...
amvisor : 2313 | vendor_central : 1271 | merged : 2550
-- Processing 2024-06 ...
amvisor : 2302 | vendor_central : 1313 | merged : 2566
-- Processing 2024-05 ...
amvisor : 2240 | vendor_central : 1330 | merged : 2522
-- Processing 2024-04 ...
amvisor : 2141 | vendor_central : 135